# Criatura Lendária 1 : Predição de Energia Total e de Formação de Nanopartículas de Cobre Utilizando Deeplearning

## Trabalho de conclusão de curso da disciplina de Redes Neurais da Ilum Escola De Ciência, em que foi proposto a criação de uma rede neural MLP para algum tema relevante no meio científico.

## Grupo 6: Anna Karen Pinto; Beatriz Borges; Paulo Henrique dos Santos
Campinas/2024

### Introdução

Materiais em escala nanométrica exibem características distintas em comparação com materiais em escalas macrométricas, devido a uma série de fatores, incluindo a superfície exposta dos materiais. Em escalas nanométricas, a relação entre área de superfície e volume é amplificada, tornando a superfície de contato proporcionalmente maior em relação ao volume do material. Essa proporção aumentada da superfície confere propriedades aos materiais nanoestruturados. [2].

Os nanomateriais possuem uma variedade de aplicações, incluindo catálise, imagiologia por ressonância magnética e liberação controlada de fármacos. Além disso, processos de modificação superficial podem ser empregados para mitigar os efeitos citotóxicos associados a certos materiais. Essas modificações podem incluir revestimentos ou funcionalizações que tornam a interação com o ambiente biológico mais favorável, reduzindo assim os efeitos adversos. São classificadas como nanométricas partículas com dimensões tipicamente entre 1-100 nm. [2][3][4]. Portanto, é fundamental entendermos como a energia total e de formação influencia no produto final e nas características para a qual a nanopartícula será designada, permitindo a implementação de medidas preventivas e o planejamento adequado, incentivando um investimento tecnológico e científico maior nesta área.

A ciência que é utilizada para esta problemática é a Rede Neural tipo MLP (multilayer perceptron) ou, em português, perceptron multicamadas, que é uma rede neural artificial moderna de alimentação direta (feedforward). Essa rede é composta por várias camadas, incluindo uma camada de entrada, uma ou mais camadas ocultas e uma camada de saída.[5].

A rede recebe os dados na camada de entrada com seus respectivos pesos. Cada neurônio possui uma função de ativação e um viez ao qual realizará cáculos. Durante o processo de aprendizado, os pesos de conexão na rede são ajustados após o processamento de cada dado com base na quantidade de erro na saída em comparação com o resultado esperado. O qual permite que um sistema aprenda e melhore de forma autônoma, sem ser programado explicitamente, alimentando-o com grandes quantidades de dados. [5]

Esses dados podem ser coletados de diversas formas possíveis, variando de acordo com sua finalidade e recursos para a pesquisa. Entretanto, algo em comum com todo qualquer tipo de dado é que eles são armazenados em dataset.

Dataset é um conjunto de dados estruturados em uma tabela, contendo descrições específicas de seus atributos e arquivos significativos para o conjunto. [6] Com o conjunto de dados, é possível extrair informações necessárias para a aplicação/manipulação desejada.

A escolha desse tipo de dataset permite uma abordagem multidisciplinar, integrando conhecimentos, sem perder a importância científica proposta pelo trabalho final.

Os códigos aqui apresetados estão organizados da seguinte forma:
- Importação das bibliotecas necessárias
- Obtenção e Tratamento dos dados
- Criação e Aplicação da MLP
- Avaliação dos Resultados e Conlusão

## Importação das Bibliotecas

Todas as bibliotecas aqui importadas foram essenciais para a realização do trabalho.

In [1]:
#Import and Treat Data
import pandas as pd
import numpy as np
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
#VIF
from sklearn.linear_model import LinearRegression
from collections import deque
import datetime
import pickle
#NN
from sklearn.metrics import mean_squared_error
import torch
import torch.nn as nn
import torch.optim as optim

## Obtenção e Tratamento dos dados

Nesta etapa, os seguintes passos foram tomados:

- Importando o Dataset e transformando em variável
- Separando em treino e teste
- Aplicando o Logaritmo e Normalização pelo máximo absoluto
- Aplicando o VIF

Cada um desses passos possui sua importância que esta explícita logo acima da sua relização em código.

### Importando o Dataset e transformando em variável:

In [2]:
df = pd.read_csv('dataset.csv', delimiter = ';')
df = df.dropna()
df

,ID,T,tau,time,N_total,N_bulk,N_surface,Volume,R_min,R_max,...,q6q6_S14,q6q6_S15,q6q6_S16,q6q6_S17,q6q6_S18,q6q6_S19,q6q6_S20,q6q6_S20+,Total_E,Formation_E
0,3281,923,45,1,133,42,91,1.570000e-27,4.737965,8.103804,...,0,0,0,0,0,0,0,0,-385.20936,85.61064
1,3282,923,45,2,136,44,92,1.600000e-27,4.023433,8.361129,...,0,0,0,0,0,0,0,0,-393.45040,87.98960
2,2291,723,50,1,149,51,98,1.750000e-27,4.571391,7.953961,...,0,0,0,0,0,0,0,0,-441.89002,85.56998
3,2292,723,50,2,151,54,97,1.780000e-27,4.754535,8.400565,...,0,0,0,0,0,0,0,0,-451.64457,82.89543
4,3091,823,50,1,170,67,103,2.000000e-27,5.673022,8.205280,...,0,0,0,0,0,0,0,0,-506.59469,95.20531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1210,523,5,10,19487,16311,3176,2.290000e-25,34.466223,44.887615,...,0,0,0,0,0,0,0,0,-65609.94800,3374.03200
3996,2110,673,5,10,19507,16289,3218,2.300000e-25,34.247301,44.673954,...,0,0,0,0,0,0,0,0,-65392.19100,3662.58900
3997,1010,423,5,10,19509,16181,3328,2.300000e-25,21.874439,46.705838,...,0,0,0,0,0,0,0,0,-66085.15400,2976.70600
3998,1910,573,5,10,19517,16255,3262,2.300000e-25,33.916720,48.546207,...,0,0,0,0,0,0,0,0,-65662.22800,3427.95200


### Separando em treino e teste

AS listas representadas abaixo são referentes, respectivamente, aos atributos e targets. Nas 3 primeiras linhas de código definimos essas listas, e nas 3 ultimas separamos entre treino e teste, sublistas que possibilitarão o treinamento e teste da rede neural. Ademais, é válido mencionar que alguns dos atributos possuíam valores zero em todos os seus dados, assim, realizamos uma seleção manual destes, e os descartamos. A lista criada abaixo já está livre destes atributos.

In [3]:
#Definição do que serão as features e os targets
X = df[["T", "tau", "time", "N_total", "N_bulk", "N_surface", "Volume", "R_min", "R_max", "R_diff", "R_avg", "R_std", "R_skew", "R_kurt", "S_100", "S_111", "S_110", "S_311", "Curve_1-10", "Curve_11-20", "Curve_21-30", "Curve_31-40", "Curve_41-50", "Curve_51-60", "Curve_61-70", "Curve_71-80", "Curve_81-90", "Curve_171-180", "Avg_total", "Avg_bulk", "Avg_surf", "TCN_1", "TCN_2", "TCN_3", "TCN_4", "TCN_5", "TCN_6", "TCN_7", "TCN_8", "TCN_9", "TCN_10", "TCN_11", "TCN_12", "TCN_13", "TCN_14", "TCN_15", "TCN_16", "BCN_6", "BCN_7", "BCN_8", "BCN_9", "BCN_10", "BCN_11", "BCN_12", "BCN_13", "BCN_14", "BCN_15", "BCN_16", "SCN_1", "SCN_2", "SCN_3", "SCN_4", "SCN_5", "SCN_6", "SCN_7", "SCN_8", "SCN_9", "SCN_10", "SCN_11", "SCN_12", "SCN_13", "SCN_14", "Avg_bonds", "Std_bonds", "Max_bonds", "Min_bonds", "N_bonds", "angle_avg", "angle_std", "FCC", "HCP", "ICOS", "DECA", "q6q6_avg_total", "q6q6_avg_bulk", "q6q6_avg_surf", "q6q6_T0", "q6q6_T1", "q6q6_T2", "q6q6_T3", "q6q6_T4", "q6q6_T5", "q6q6_T6", "q6q6_T7", "q6q6_T8", "q6q6_T9", "q6q6_T10", "q6q6_T11", "q6q6_T12", "q6q6_T13", "q6q6_T14", "q6q6_B0", "q6q6_B1", "q6q6_B2", "q6q6_B3", "q6q6_B4", "q6q6_B5", "q6q6_B6", "q6q6_B7", "q6q6_B8", "q6q6_B9", "q6q6_B10", "q6q6_B11", "q6q6_B12", "q6q6_B13", "q6q6_B14", "q6q6_B15", "q6q6_B16", "q6q6_B17", "q6q6_B18", "q6q6_B19", "q6q6_B20", "q6q6_B20+", "q6q6_S0", "q6q6_S1", "q6q6_S2", "q6q6_S3", "q6q6_S4", "q6q6_S5", "q6q6_S6", "q6q6_S7", "q6q6_S8", "q6q6_S9", "q6q6_S10", "q6q6_S11", "q6q6_S12", "q6q6_S13", "q6q6_S20+"]]
y = df[["Total_E", "Formation_E"]]
atributos = ["T", "tau", "time", "N_total", "N_bulk", "N_surface", "Volume", "R_min", "R_max", "R_diff", "R_avg", "R_std", "R_skew", "R_kurt", "S_100", "S_111", "S_110", "S_311", "Curve_1-10", "Curve_11-20", "Curve_21-30", "Curve_31-40", "Curve_41-50", "Curve_51-60", "Curve_61-70", "Curve_71-80", "Curve_81-90", "Curve_171-180", "Avg_total", "Avg_bulk", "Avg_surf", "TCN_1", "TCN_2", "TCN_3", "TCN_4", "TCN_5", "TCN_6", "TCN_7", "TCN_8", "TCN_9", "TCN_10", "TCN_11", "TCN_12", "TCN_13", "TCN_14", "TCN_15", "TCN_16", "BCN_6", "BCN_7", "BCN_8", "BCN_9", "BCN_10", "BCN_11", "BCN_12", "BCN_13", "BCN_14", "BCN_15", "BCN_16", "SCN_1", "SCN_2", "SCN_3", "SCN_4", "SCN_5", "SCN_6", "SCN_7", "SCN_8", "SCN_9", "SCN_10", "SCN_11", "SCN_12", "SCN_13", "SCN_14", "Avg_bonds", "Std_bonds", "Max_bonds", "Min_bonds", "N_bonds", "angle_avg", "angle_std", "FCC", "HCP", "ICOS", "DECA", "q6q6_avg_total", "q6q6_avg_bulk", "q6q6_avg_surf", "q6q6_T0", "q6q6_T1", "q6q6_T2", "q6q6_T3", "q6q6_T4", "q6q6_T5", "q6q6_T6", "q6q6_T7", "q6q6_T8", "q6q6_T9", "q6q6_T10", "q6q6_T11", "q6q6_T12", "q6q6_T13", "q6q6_T14", "q6q6_B0", "q6q6_B1", "q6q6_B2", "q6q6_B3", "q6q6_B4", "q6q6_B5", "q6q6_B6", "q6q6_B7", "q6q6_B8", "q6q6_B9", "q6q6_B10", "q6q6_B11", "q6q6_B12", "q6q6_B13", "q6q6_B14", "q6q6_B15", "q6q6_B16", "q6q6_B17", "q6q6_B18", "q6q6_B19", "q6q6_B20", "q6q6_B20+", "q6q6_S0", "q6q6_S1", "q6q6_S2", "q6q6_S3", "q6q6_S4", "q6q6_S5", "q6q6_S6", "q6q6_S7", "q6q6_S8", "q6q6_S9", "q6q6_S10", "q6q6_S11", "q6q6_S12", "q6q6_S13", "q6q6_S20+"]

# Split de treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.1, random_state=10)
y_treino = y_treino.dropna()      #Retirada de valores NAN
X_treino = X_treino.dropna()      #Retirada de valores NAN

### Aplicando o Logaritmo e Normalização pelo máximo absoluto

Foi proposto em em sala com o Orientador que nosso dados precisariam passar por duas normalizações principais: logaritmica e pelo máximo absoluto. Entretanto, nossos dados possuíam muitos valores 0, e o resultado do logaritmo de 0 é uma indefinição matemática. Na biblioteca que utilizamos para realizar o nosso logaritmo(linhas de código comentadas abaixo), o resultado do log de 0 é um valor NaN. Por isso, estávamos encontrando problemas ao treinar a rede, uma vez que o dataset X possuía uma quantidade de dados diferente do Y. Dessa forma, o grupo decidiu por não utilizar o logaritmo.

Assim, realizmaos apenas a normalização pelo máximo absoluto, que é um procedimento utilizado para escalar os valores de um conjunto de dados de forma que o valor máximo seja 1 (ou -1, dependendo do intervalo escolhido). O processo em si é bastante simples e envolve dividir todos os valores do conjunto de dados pelo valor máximo absoluto encontrado no conjunto.

In [4]:
#[6]

#X_treino_log = np.log(np.array(X_treino))
#X_teste_log = np.log(np.array(X_teste))
#X_treino_log = X_treino_log.dropna()

max_abs_normalizador = MaxAbsScaler()     #Definindo o normalizador
X_treino_log_normalizado = max_abs_normalizador.fit_transform(X_treino)     #Normalizando X
X_teste_log_normalizado = max_abs_normalizador.fit_transform(X_teste)     #Normalizando o Y

### Aplicando o VIF

Nosso dataset era composto por muitos atributos, e uma possível mitigação para este problema seria aplicar um VIF, que significa "Variance Inflation Factor" (Fator de Inflação da Variância). É uma medida estatística usada para diagnosticar multicolinearidade em uma regressão linear. Multicolinearidade ocorre quando duas ou mais variáveis independentes em um modelo de regressão linear estão altamente correlacionadas, o que pode causar problemas na interpretação dos coeficientes de regressão e na precisão das previsões.[7]

Os códigos deste algorítmo foram disponibilizados pelo professor, de um acervo próprio:

In [5]:
def vif_selection(
    x: np.ndarray,
    cols: list,
    max_vif: float = 8,
    deleted_cols=[],
    savepath=None,
    verbose=False,
    initial_pass=False,
):

    min_tol = 1 / max_vif
    idx = 0

    def quick_tolerance_min_idx(x, idx=0):
        tolerance = np.zeros(x.shape[1])

        # to improve speed of finding the intial zeros
        gen = deque(range(x.shape[1]))
        gen.rotate(-idx)
        gen = list(gen)

        for i in gen:
            X, y = np.delete(x, i, 1), x[:, i]
            # https://stackoverflow.com/questions/36573046/difference-between-numpy-linalg-lstsq-and-sklearn-linear-model-linearregression
            r_squared = LinearRegression().fit(X, y).score(X, y)
            tol = 1 - r_squared
            if tol == 0:
                return i, tol
            else:
                tolerance[i] = tol

        idx = np.argmin(tolerance)
        return idx, tolerance[idx]

    if verbose:
        print(len(cols), datetime.datetime.now())

    def check(tol):
        if initial_pass:
            return tol == 0
        else:
            return tol < min_tol

    while True:
        idx, tol = quick_tolerance_min_idx(x, idx)
        if check(tol):
            x = np.delete(x, idx, 1)
            poped_col = cols.pop(idx)
            deleted_cols.extend([poped_col])
            if savepath:
                pickle.dump(deleted_cols, open(savepath, "wb"))

            if verbose:
                print(
                    len(cols), datetime.datetime.now(), f"{tol:.3g}", poped_col
                )
        else:
            break

    return x, cols, deleted_cols

In [6]:
X_norm_vif = vif_selection(X_treino_log_normalizado, atributos)     #Aplicação do VIF em nossos dados
X_final, cols, deleted_cols = X_norm_vif      #Separando o resultado do VIF em 3 dados: dataset final, colunas totais e colunas apagadas

Após todas estas etapas, nossos dados estão prontos para serem aplicados na MLP.

## Criação e Aplicação da MLP

Para a criação e aplicação da MLP utilizamos como base o caderno da aula de Torch, mininstrada pelo nosso Orientador. Assim, realizamos algumas mudanças para adaptar a MLP ao nosso caso.[8] 

Precisávamos de uma rede que variasse a quantidade de neurônios e de camadas, para que encontrássemos a melhor configuração possível para nossos dados.

In [7]:
class MLP(nn.Module):

    def __init__(self, num_dados_entrada, neuronios_camadas, num_targets):
        super().__init__()
        
        layers = []      # Lista para armazenar as camadas
        
        for i in range(len(neuronios_camadas) - 1): # Adicionando camadas intermediárias
            layers.append(nn.Linear(neuronios_camadas[i], neuronios_camadas[i+1]))
            layers.append(nn.Sigmoid())
        
        layers.append(nn.Linear(neuronios_camadas[-1], num_targets))     # Adicionando camada de saída
        
        self.camadas = nn.Sequential(*layers)     # Definindo as camadas como uma sequência
        
    def forward(self, x):
        x = x.float()
        x = self.camadas(x)
        return x
    
x = torch.tensor(X_final)     #Transformando nossos dados em tensores, formato utilizado pelo Pytorch
y = torch.tensor(np.array(y_treino))
y = y.float()

Agora que  temos nossa rede criada, precisamos definir o intervalo de variação dos nosso hiperparâmetros, buscando uma alternativa que não seja muito custosa computacionalmente mas que ainda assim atinja nossas espectativas.

In [9]:
NUM_DADOS_DE_ENTRADA = x.shape[1]     #Numero de dados de entradas representado pela quantidade de colunas do dataset

NUM_DADOS_DE_SAIDA = 2     #Número de dados de saída

NEURONIOS = list(range(10, 80, 2))     #Intervalo da quantidade de neurônios

CAMADAS = list(range(2, 6))     #Intervalo da quantidade de camadas

TAXA_DE_APRENDIZADO = 0.01      #Definindo a Taxa de Aprendizado da MLP
 
#otimizador = optim.SGD(minha_mlp.parameters(), lr=TAXA_DE_APRENDIZADO)     #Definindo o Otimizador
        
fn_perda = nn.MSELoss()     #Definindo a função Perda
        
NUM_EPOCAS = 10000     #Definindo o número de épocas

Com os hiperparâmetros definidos, criamos um loop que irá iterar de forma a criar e testar várias redes, vairando a quantidade de camadas e neurônios como propomos. Para isso, criamos duas variáveis, uma que armazenará os hiperparâmetros(a rede sem si) e outra que armazenará o valor do MSE(função de perda adotada pelo grupo). Assim, sempre que uma nova rede for criada, o valor do melhor MSE dessa rede vai ser comparado com o armazenado na variável fixa, caso esse valor seja menor que o lá armazeado, ele o substituirá. Assim, a rede com o valor menor também irá substituir a rede já colocada na variável que armazena os hiperparâmetros.

In [10]:
melhor_mse = float('inf')     #Criando a variável do MSE
melhor_rede = None     #Criando a variável dos hiperparâmetros

for layer in CAMADAS:    #Criando o loop que itera as camadas
    
    for neuronios in NEURONIOS:     #Criando o loop que itera os neurônios
        neuronios_camadas = [NUM_DADOS_DE_ENTRADA]     #Criando uma lista com o valor dos dados de entrada
        
        for a in range(layer):
            neuronios_camadas.append(neuronios)     #Adicionando os neurônios em cada camada na quantidade atual do loop
        
        minha_mlp = MLP(     #Criando a instância da classe da MLP
            NUM_DADOS_DE_ENTRADA, neuronios_camadas, NUM_DADOS_DE_SAIDA
        )
        
        otimizador = optim.SGD(minha_mlp.parameters(), lr=TAXA_DE_APRENDIZADO)     #Definindo o Otimizador
       
        minha_mlp.train()     #Definindo a ação para treino da MLP

        for epoca in range(NUM_EPOCAS):      #Treinando a MLP, passando por cada passo
            
            y_pred = minha_mlp(x)     #Forward pass

            otimizador.zero_grad()     #Zero grad

            loss = fn_perda(y, y_pred)     #Loss
            
            loss.backward()     #Backpropagation

            otimizador.step()     #Atualiza parâmetros

            if loss < melhor_mse:     #Verifica se o MSE atual é melhor do que o melhor MSE registrado até agora
                melhor_mse = loss
                melhor_rede = minha_mlp

with open('melhor_rede.pkl', 'wb') as f:     #Salva a melhor rede em um arquivo usando pickle
    pickle.dump(melhor_rede, f)

print("Melhor MSE:", melhor_mse)

Melhor MSE: tensor(89030672., grad_fn=<MseLossBackward0>)


## Avaliação dos Resultados e Conclusão

# Referências

[1] Copper Nanoparticle Data Set. Disponível em: https://data.csiro.au/collection/csiro:42598. Acesso em: 09 abr. 2024.

[2] OS NANOMATERIAIS E A DESCOBERTA DE NOVOS MUNDOS NA BANCADA DO QUÍMICO | Manuel A. Martins e Tito Trindade - Quim. Nova, Vol. 35, No. 7, 1434-1446, 2012. Disponível em: https://www.scielo.br/j/qn/a/P8tgywDnt7nS6tGyHdQ3BCF/. Acesso em: 02 mai. 2024.

[3] Ojha, N. K.; Zyryanov, G. V.; Majee, A.; Charushin, V. N.; Chupakhin, O. N.; Santra, S. Copper nanoparticles as inexpensive and efficient catalyst: A valuable contribution inorganic synthesis. Coordination Chemistry Reviews 2017, 353, 1–57.11.

‌[4] Ssekatawa K, Byarugaba DK, Angwe MK, Wampande EM, Ejobi F, Nxumalo E, Maaza M, Sackey J, Kirabira JB. Phyto-Mediated Copper Oxide Nanoparticles for Antibacterial, Antioxidant and Photocatalytic Performances. Front Bioeng Biotechnol. 2022 Feb 16;10:820218. doi: 10.3389/fbioe.2022.820218. PMID: 35252130; PMCID: PMC8889028.

‌[5] Multilayer perceptron | Wikipedia, the free encyclopedia. Disponível em https://en.wikipedia.org/wiki/Multilayer_perceptron#:~:text=A%20multilayer%20perceptron%20(MLP)%20is,that%20is%20not%20linearly%20separable.. Acesso em: 29 abr. 2024.

[6] https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MaxAbsScaler.html

[7] https://online.stat.psu.edu/stat462/node/180/

[8] ATP-303 NN 5.2 - Notebook PyTorch